In [1]:
# importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import sklearn 
import numpy as np

In [2]:
# reading dataframe from csv file
df = pd.read_csv("final_data_with_slopes.csv")

In [3]:
target_saps_I = df["SAPS-I"]
target_SOFA = df["SOFA"]
target_survival = df["Survival"]

In [4]:
# removing unnecessary columns.
df = df.drop(columns=['Height_last','Height_max','Height_med','Height_min','Height_mode','Height_slope','Height_std',
'Gender_last','Gender_max','Gender_med','Gender_min','Gender_mode','Gender_slope','Gender_std',
'Age_last','Age_max','Age_med','Age_min','Age_mode','Age_slope','Age_std','Unnamed: 0',
'SAPS-I','SOFA','Length_of_stay','Survival','ICUType_first','ICUType_last','ICUType_max','ICUType_med','ICUType_min','ICUType_mode','ICUType_slope','ICUType_std'])

In [5]:
record_col = df["RecordID"]
df = df.drop(columns=['RecordID'])

In [6]:
# separating target column from dataframe
target_col = df["In-hospital_death"]
df = df.drop(columns=['In-hospital_death'])

In [7]:
# Create correlation matrix with absolute values
corr_matrix = df.corr()

In [8]:
import statsmodels.api as sm
from patsy import dmatrices

In [16]:
mod = sm.OLS(target_saps_I, df)  

res = mod.fit()  

In [17]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 SAPS-I   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     20.20
Date:                Wed, 07 Aug 2019   Prob (F-statistic):               0.00
Time:                        23:04:47   Log-Likelihood:                -11214.
No. Observations:                4000   AIC:                         2.292e+04
Df Residuals:                    3753   BIC:                         2.448e+04
Df Model:                         246                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
ALP_first            -0.0075      0.00

In [15]:
# Create correlation matrix with absolute values
corr_matrix = df.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
df = df.drop(df[to_drop], axis=1)

In [18]:
#After removing correlated variables
mod = sm.OLS(target_saps_I, df)  
res = mod.fit()  

In [19]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 SAPS-I   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     20.20
Date:                Wed, 07 Aug 2019   Prob (F-statistic):               0.00
Time:                        23:05:31   Log-Likelihood:                -11214.
No. Observations:                4000   AIC:                         2.292e+04
Df Residuals:                    3753   BIC:                         2.448e+04
Df Model:                         246                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
ALP_first            -0.0075      0.00

In [20]:
res.params

ALP_first           -0.007540
ALP_last             0.007688
ALP_slope           -0.040081
ALP_std              0.003997
ALT_first            0.001685
ALT_last             0.001870
ALT_med              0.000261
ALT_min             -0.004193
ALT_slope           -0.051701
ALT_std             -0.002363
AST_first           -0.000094
AST_last            -0.001360
AST_max             -0.000555
AST_med             -0.000812
AST_min              0.002860
AST_slope            0.034837
AST_std              0.002648
Age_first            0.079308
Albumin_first       -0.417773
Albumin_last        -0.036574
Albumin_slope       -2.167381
Albumin_std          0.821525
BUN_first            0.007901
BUN_last             0.000236
BUN_slope            1.954574
BUN_std              0.031922
Bilirubin_first      0.090185
Bilirubin_slope     -0.568172
Bilirubin_std        0.015330
Cholesterol_first   -0.001764
                       ...   
TroponinI_std        0.392917
TroponinT_first     -0.007230
TroponinT_

In [21]:
res.rsquared

0.5696742589511332

In [23]:
sm.stats.linear_rainbow(res)

(1.1661171004351925, 0.00046198049047804795)

In [78]:
cols = (res.pvalues < 0.05)

In [114]:
significant_cols = cols.where(cols == True).to_frame().dropna().index.tolist()

In [116]:
significant_df = df[significant_cols]